In [1]:
import os
import pickle
import pandas as pd
from tokenizer import NLTKTokenizer

CWD = os.getcwd()
TRAIN_DATA_PATH = os.path.join(CWD, 'data', 'trainset.csv')
VALID_DATA_PATH = os.path.join(CWD, 'data', 'validset.csv')
TEST_DATA_PATH = os.path.join(CWD, 'data', 'testset.csv')
DICT_PATH = os.path.join(CWD, 'data', 'dictionary.pkl')
WORKERS = os.cpu_count() // 2
Tokenizer = NLTKTokenizer()

In [2]:
import numpy as np

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [4]:
def SplitSent(doc):
    return doc.split('$$$')


def GenDict(train, valid):
    global Tokenizer
    if os.path.exists(DICT_PATH):
        Tokenizer = NLTKTokenizer.load_from_file(DICT_PATH)
    else:
        for item in train['Abstract']:
            Tokenizer.build_dict(item)

        for item in valid['Abstract']:
            Tokenizer.build_dict(item)

        Tokenizer.save_to_file(DICT_PATH)

In [5]:
train = pd.read_csv(TRAIN_DATA_PATH)
valid = pd.read_csv(VALID_DATA_PATH)
test = pd.read_csv(TEST_DATA_PATH)

train['Abstract'] = train['Abstract'].apply(func=SplitSent)
valid['Abstract'] = valid['Abstract'].apply(func=SplitSent)
GenDict(train, valid)

In [6]:
test['Abstract'] = test['Abstract'].apply(func=SplitSent)

In [7]:
def labels_to_onehot(labels):
    '''
    Convert labels to one-hot encoding

    Args : 
        labels:( DataFrame column item ) 
    Return :
        one_hot_labels: ( DataFrame column item )
    '''
    one_hot_labels = []
    label_list = labels.split(' ')
    label_dict = {'BACKGROUND': 0, 'OBJECTIVES':1, 'METHODS':2, 'RESULTS':3, 'CONCLUSIONS':4, 'OTHERS':5}
    for label in label_list:
        onehot = [0,0,0,0,0,0]
        for l in label.split('/'):
            onehot[label_dict[l]] = 1
        one_hot_labels.append(onehot)
    
    return one_hot_labels


In [8]:
def encode_data(dataset):
    '''
    encode 'Abstract' and convert label to one_hot


    Args:
        dataset(pd.DataFrame)
    '''
    global Tokenizer
    dataset['Abstract'] = dataset['Abstract'].apply(func=Tokenizer.encode)
    if 'Task 1' in dataset.columns:
        dataset['Task 1'] = dataset['Task 1'].apply(func=labels_to_onehot)

In [9]:
encode_data(train)
encode_data(valid)
encode_data(test)

In [10]:
train.iloc[0]

Abstract    [[42485, 35732, 42367, 42450, 38674, 10, 42508...
Task 1      [[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 1...
Name: 0, dtype: object

In [11]:
class Abstract(Dataset):
    def __init__(self, data, pad_idx, eos_token):
        self.data = data
        self.pad_idx = pad_idx
        self.eos_token = eos_token
    
    def __len__(self):
        return len(self.data.index)
    
    def __getitem__(self, index):
        return self.data.iloc[index]
    
    def collate_fn(self, datas):
        abstracts = [ torch.as_tensor( abstract, dtype = torch.long ) for data in datas for abstract in data['Abstract'] ]
        batch_abstracts = pad_sequence( abstracts, batch_first = True, padding_value = self.pad_idx )

        labels = [ torch.as_tensor( label, dtype = torch.float) for data in datas for label in data['Task 1'] ]
        batch_labels = pad_sequence( labels, batch_first = True, padding_value = self.pad_idx )



In [12]:
dataset = Abstract(data=train, pad_idx=0)

In [13]:
dataset[0]

Abstract    [[42485, 35732, 42367, 42450, 38674, 10, 42508...
Task 1      [[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 1...
Name: 0, dtype: object

In [42]:
labels = [ torch.as_tensor(abstract, dtype=torch.float) for abstract in dataset[0]['Task 1'] ]

In [43]:
labels

[tensor([1., 0., 0., 0., 0., 0.]),
 tensor([1., 0., 0., 0., 0., 0.]),
 tensor([1., 1., 0., 0., 0., 0.]),
 tensor([0., 0., 1., 0., 0., 0.]),
 tensor([0., 0., 1., 0., 0., 0.]),
 tensor([0., 0., 1., 1., 0., 0.])]

In [45]:
batch_labels = pad_sequence( labels, batch_first=True )
batch_labels

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0.]])

In [33]:
sequence = [ torch.as_tensor(vec) for  vec in dataset[0]['Abstract'] ]

In [34]:
sequence

[tensor([42485, 35732, 42367, 42450, 38674,    10, 42508, 42313, 42472, 38387,
         42474, 42459, 42455, 42471, 42228, 42500, 42356, 42451, 42463, 42441,
         42463, 42416, 42306, 42468, 42437, 42396, 42445, 42526, 42450, 42105,
         42115, 42451, 42550, 42366, 42465,     3]),
 tensor([42007, 42500, 42497, 42204, 42502, 42035, 42438, 42453, 42451, 42497,
         42514, 42502, 42531, 41078, 42556, 42515, 40045, 42459, 42452, 42497,
         40188, 41599, 42465,     3]),
 tensor([41532, 42488, 42453, 42351, 42451, 42455, 42404, 42502, 42541, 42555,
         42445, 42515, 40045, 42507, 42432, 42495, 42500, 42435, 42416, 42463,
         42516, 42468, 42531, 42541, 42499, 41271, 42497, 42484, 42481, 42455,
         42360, 42476, 42465,     3]),
 tensor([42516, 42453, 42518, 42451, 42519, 42396, 42455, 42517, 42481, 42424,
         42515, 40045, 42451, 42466,    80, 42451, 42469, 42459, 42407, 42445,
         42396, 42452, 42455, 41566, 42299, 42502, 42436, 42451, 42474, 42424,


In [35]:
sequence[0].size()

torch.Size([36])

In [36]:
pad = pad_sequence(sequence, batch_first=True)
pad

tensor([[42485, 35732, 42367, 42450, 38674,    10, 42508, 42313, 42472, 38387,
         42474, 42459, 42455, 42471, 42228, 42500, 42356, 42451, 42463, 42441,
         42463, 42416, 42306, 42468, 42437, 42396, 42445, 42526, 42450, 42105,
         42115, 42451, 42550, 42366, 42465,     3,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [42007, 42500, 42497, 42204, 42502, 42035, 42438, 42453, 42451, 42497,
         42514, 42502, 42531, 41078, 42556, 42515, 40045, 42459, 42452, 42497,
         40188, 41599, 42465,     3,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [41532, 42488, 42453, 42351, 42451, 42455, 42404, 42502, 42541, 42555,
         42445, 42515, 40045, 42507, 42432, 42495, 42500, 42435, 4241

In [37]:
pad.size()

torch.Size([6, 53])

In [38]:
loader = DataLoader(dataset = dataset, batch_size = 2, collate_fn=dataset.collate_fn)